In [2]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import pickle

In [2]:
df = pd.read_csv('NOTEEVENTS.csv')

/Users/el/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2083180 entries, 0 to 2083179
Data columns (total 11 columns):
 #   Column       Dtype  
---  ------       -----  
 0   ROW_ID       int64  
 1   SUBJECT_ID   int64  
 2   HADM_ID      float64
 3   CHARTDATE    object 
 4   CHARTTIME    object 
 5   STORETIME    object 
 6   CATEGORY     object 
 7   DESCRIPTION  object 
 8   CGID         float64
 9   ISERROR      float64
 10  TEXT         object 
dtypes: float64(3), int64(2), object(6)
memory usage: 174.8+ MB


In [4]:
df.groupby(["CATEGORY"]).count()

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,DESCRIPTION,CGID,ISERROR,TEXT
CATEGORY,,,,,,,,,,
Case Management,967,967,954,967,967,967,967,967,14,967
Consult,98,98,98,98,98,98,98,98,0,98
Discharge summary,59652,59652,59652,59652,0,0,59652,0,0,59652
ECG,209051,209051,138190,209051,0,0,209051,0,0,209051
Echo,45794,45794,34037,45794,0,0,45794,0,0,45794
General,8301,8301,8209,8301,8260,8301,8301,8301,65,8301
Nursing,223556,223556,220758,223556,222172,223556,223556,223556,374,223556
Nursing/other,822497,822497,821258,822497,822497,822497,822497,822497,0,822497
Nutrition,9418,9418,9378,9418,9411,9418,9418,9418,18,9418


In [5]:
# Discard discharge summaries as in Chen et al. 2018
df = df[df["CATEGORY"] != "Discharge summary"]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2023528 entries, 52108 to 2083179
Data columns (total 11 columns):
 #   Column       Dtype  
---  ------       -----  
 0   ROW_ID       int64  
 1   SUBJECT_ID   int64  
 2   HADM_ID      float64
 3   CHARTDATE    object 
 4   CHARTTIME    object 
 5   STORETIME    object 
 6   CATEGORY     object 
 7   DESCRIPTION  object 
 8   CGID         float64
 9   ISERROR      float64
 10  TEXT         object 
dtypes: float64(3), int64(2), object(6)
memory usage: 185.3+ MB


In [6]:
df.groupby(["CATEGORY"]).count()

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,DESCRIPTION,CGID,ISERROR,TEXT
CATEGORY,,,,,,,,,,
Case Management,967,967,954,967,967,967,967,967,14,967
Consult,98,98,98,98,98,98,98,98,0,98
ECG,209051,209051,138190,209051,0,0,209051,0,0,209051
Echo,45794,45794,34037,45794,0,0,45794,0,0,45794
General,8301,8301,8209,8301,8260,8301,8301,8301,65,8301
Nursing,223556,223556,220758,223556,222172,223556,223556,223556,374,223556
Nursing/other,822497,822497,821258,822497,822497,822497,822497,822497,0,822497
Nutrition,9418,9418,9378,9418,9411,9418,9418,9418,18,9418
Pharmacy,103,103,102,103,102,103,103,103,2,103


In [6]:
def clean_text(text):
    new_text = text.lower()
    new_text = re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", " ", new_text)
    # remove numbers
    new_text = re.sub(r"\d+", "", new_text)
    new_text = " ".join(new_text.split())
    
    return new_text

# Clean text
clean_notes = dict()
for index, row in df.iterrows():
    subject_id = row["SUBJECT_ID"]
    tokens = clean_text(row["TEXT"])
    if subject_id not in clean_notes.keys():
        clean_notes[subject_id] = tokens
    else:
        clean_notes[subject_id] += " " +  tokens

clean_df = pd.DataFrame({"SUBJECT_ID": list(clean_notes.keys()), "TEXT": list(clean_notes.values())})
# Tfidf
matrix = TfidfVectorizer(stop_words="english", ngram_range=(1,1),
              max_features=10000, use_idf=True).fit_transform(clean_df["TEXT"])
# Store
pickle.dump([clean_df["SUBJECT_ID"], matrix], open("processed/intermediate_features.pkl", "wb"))

'\n# Clean text\nclean_notes = dict()\nfor index, row in df.iterrows():\n    subject_id = row["SUBJECT_ID"]\n    tokens = clean_text(row["TEXT"]).split(" ")\n    if subject_id not in clean_notes.keys():\n        clean_notes[subject_id] = tokens\n    else:\n        clean_notes[subject_id].append(*tokens)\n'

In [11]:
df_patients = pd.read_csv("../PATIENTS.csv")
df_admissions = pd.read_csv("../ADMISSIONS.csv")

patient_deaths = dict()
patient_sex = dict()
patient_race = dict()

for index, row in df_patients.iterrows():
    subject_id = row["SUBJECT_ID"]
    patient_deaths[subject_id] = row["EXPIRE_FLAG"]
    patient_sex[subject_id] = row["GENDER"]
    patient_race[subject_id] = df_admissions[(df_admissions["SUBJECT_ID"] == subject_id)]["ETHNICITY"].values[0]

In [49]:
subject_ids, feature_matrix = pickle.load(open("processed/intermediate_features.pkl", "rb"))

In [60]:
labels = {
    "SUBJECT_ID": list(), "IS_DEAD": list(), 
    "IS_RACE_BLACK": list(), "IS_RACE_WHITE": list(), 
    "IS_RACE_HISPANIC": list(), "IS_RACE_ASIAN": list(),
    "IS_RACE_OTHER": list(),
    "IS_SEX_M": list(), "IS_SEX_F": list()
    }
for s_id in subject_ids:
    labels["SUBJECT_ID"].append(s_id)
    labels["IS_DEAD"].append(patient_deaths[s_id])
    if "BLACK" in patient_race[subject_id]:
        labels["IS_RACE_BLACK"].append(1)
        labels["IS_RACE_HISPANIC"].append(0)
        labels["IS_RACE_WHITE"].append(0)
        labels["IS_RACE_ASIAN"].append(0)
        labels["IS_RACE_OTHERS"].append(0)
    elif "WHITE" in patient_race[subject_id]:
        labels["IS_RACE_BLACK"].append(0)
        labels["IS_RACE_HISPANIC"].append(0)
        labels["IS_RACE_WHITE"].append(1)
        labels["IS_RACE_ASIAN"].append(0)
        labels["IS_RACE_OTHERS"].append(0)
    elif "ASIAN" in patient_race[subject_id]:
        labels["IS_RACE_BLACK"].append(0)
        labels["IS_RACE_HISPANIC"].append(0)
        labels["IS_RACE_WHITE"].append(0)
        labels["IS_RACE_ASIAN"].append(1)
        labels["IS_RACE_OTHERS"].append(0)
    if "HISPANIC" in patient_race[subject_id]:
        labels["IS_RACE_BLACK"].append(0)
        labels["IS_RACE_HISPANIC"].append(1)
        labels["IS_RACE_WHITE"].append(0)
        labels["IS_RACE_ASIAN"].append(0)
        labels["IS_RACE_OTHERS"].append(0)
    else:
        labels["IS_RACE_BLACK"].append(0)
        labels["IS_RACE_HISPANIC"].append(0)
        labels["IS_RACE_WHITE"].append(0)
        labels["IS_RACE_ASIAN"].append(0)
        labels["IS_RACE_OTHERS"].append(1)

    if patient_sex[subject_id] is "F":
        labels["IS_SEX_F"].append(1)
        labels["IS_SEX_M"].append(0)
    else:
        labels["IS_SEX_F"].append(0)
        labels["IS_SEX_M"].append(1)


In [69]:
df_labels = pd.DataFrame(labels)
df_labels.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46047 entries, 0 to 46046
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   SUBJECT_ID  46047 non-null  int64
 1   IS_DEAD     46047 non-null  int64
dtypes: int64(2)
memory usage: 719.6 KB


In [73]:
pickle.dump([feature_matrix, list(df_labels["IS_DEAD"]), list(df_labels["SUBJECT_ID"])], 
            open("processed/mimic_tfidf.pkl", "wb"))

In [14]:
df_patients.groupby("GENDER").count()

,ROW_ID,SUBJECT_ID,DOB,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG
GENDER,,,,,,,
F,20399,20399,20399,7235,4548,6112,20399
M,26121,26121,26121,8524,5426,7266,26121


In [16]:
df_admissions.groupby("ETHNICITY").count()

,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA
ETHNICITY,,,,,,,,,,,,,,,,,,
AMERICAN INDIAN/ALASKA NATIVE,51,51,51,51,51,3,51,51,51,51,16,51,22,14,14,51,51,51
AMERICAN INDIAN/ALASKA NATIVE FEDERALLY RECOGNIZED TRIBE,3,3,3,3,3,1,3,3,3,3,3,3,3,2,2,3,3,3
ASIAN,1509,1509,1509,1509,1509,105,1509,1509,1509,1509,665,1502,781,510,510,1507,1509,1509
ASIAN - ASIAN INDIAN,85,85,85,85,85,3,85,85,85,85,77,85,81,44,44,85,85,85
ASIAN - CAMBODIAN,17,17,17,17,17,4,17,17,17,17,17,17,17,11,11,17,17,17
ASIAN - CHINESE,277,277,277,277,277,27,277,277,277,277,247,277,247,164,164,277,277,277
ASIAN - FILIPINO,25,25,25,25,25,2,25,25,25,25,23,25,23,19,19,25,25,25
ASIAN - JAPANESE,7,7,7,7,7,2,7,7,7,7,6,7,6,3,3,7,7,7
ASIAN - KOREAN,13,13,13,13,13,1,13,13,13,13,12,13,12,7,7,13,13,13


In [27]:
patient_sex[250] == "F"

True